ETL 

We will start with the ETL stage. Unlike the EDA, where only an exploration of the data was performed, here in the ETL, is where the relevant changes are made, so that at the end you have a reliable database, columns that are not appropriate for the type of query to be performed in the api will be eliminated. This in order to consolidate data that is easy to understand. 

We loaded all the necessary notebooks for the ETL process.

In [95]:
import pandas as pd
import numpy as np

In [96]:
df_amazon = pd.read_csv('Datasets/amazon_prime_titles-score.csv')
df_disney = pd.read_csv('Datasets/disney_plus_titles-score.csv')
df_hulu = pd.read_csv('Datasets/hulu_titles-score (2).csv')
df_netflix = pd.read_csv('Datasets/netflix_titles-score.csv')

Each id will consist of the first letter of the platform name, followed by the show_id already present in the datasets.

In [97]:
df_amazon['show_id'] = "a" + df_amazon.show_id 
df_disney['show_id'] = "d" + df_disney.show_id 
df_hulu['show_id'] = "h" + df_hulu.show_id 
df_netflix['show_id'] = "n" + df_netflix.show_id 

All dataframes are merged into one, for better management.

In [98]:
data_string = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], axis=0)

Rename columns. 

In [99]:
cambios={'show_id': 'id',
        'type':'category', 
         'title':'title',
         'director':'director',
         'cast':'cast',
         'country':'country',
         'date_added':'date_added',
         'release_year':'release_year',
         'rating':'rating',
         'duration':'duration',
         'listed_in':'genre',
         'description':'synopsis'
         }

data_string.rename(columns = cambios, inplace=True)

Remove spaces at the sides of the "date_added" column and format "date_added".

In [100]:
data_string["date_added"] = data_string["date_added"].str.lstrip()
data_string["date_added"] = data_string["date_added"].str.lstrip()
data_string["date_added"]=data_string["date_added"].str.replace(",", "")
data_string["date_added"] = pd.to_datetime(data_string["date_added"], format= "%B %d %Y")

Replace the null values in the rating column by the string "G".

In [101]:
data_string.fillna({'rating': "G"}, inplace=True)

In [102]:
data_string["id"] = data_string["id"].str.lower()
data_string["category"] = data_string["category"].str.lower()
data_string["title"] = data_string["title"].str.lower()
data_string["director"] = data_string["director"].str.lower()
data_string["cast"] = data_string["cast"].str.lower()
data_string["country"] = data_string["country"].str.lower()
data_string["duration"] = data_string["duration"].str.lower()
data_string["genre"] = data_string["genre"].str.lower()
data_string["synopsis"] = data_string["synopsis"].str.lower()


The numerical part of the string is separated in the column "duration". 

In [103]:
separated = data_string["duration"].str.split(' ', n=1, expand=True)
data_string["duration_int"] = separated[0]
data_string["duration_type"] = separated[1]
data_string.drop(['duration'], axis=1)

,id,category,title,director,cast,country,date_added,release_year,rating,genre,synopsis,score,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,G,"comedy, drama",a small fishing village must procure a local d...,99,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,37,110,min
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,G,"action, drama, suspense",after a man discovers his wife is cheating on ...,20,74,min
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,G,documentary,"pink breaks the mold once again, bringing her ...",27,69,min
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,G,"drama, fantasy",teenage matt banting wants to work with a famo...,75,45,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,ns8803,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,R,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",20,158,min
8803,ns8804,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,TV-Y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",8,2,seasons
8804,ns8805,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,R,"comedies, horror movies",looking to survive in a world taken over by zo...,55,88,min
8805,ns8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,PG,"children & family movies, comedies","dragged from civilian life, a former superhero...",7,88,min


In the column "duration_type" we have several records "seasons" will be replaced by "season".

In [104]:
data_string["duration_type"].replace({"seasons":"season"}, inplace=True)

The "duration" column is eliminated.

In [106]:
data_string.drop(["duration"], axis=1)

,id,category,title,director,cast,country,date_added,release_year,rating,genre,synopsis,score,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,G,"comedy, drama",a small fishing village must procure a local d...,99,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,37,110,min
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,G,"action, drama, suspense",after a man discovers his wife is cheating on ...,20,74,min
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,G,documentary,"pink breaks the mold once again, bringing her ...",27,69,min
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,G,"drama, fantasy",teenage matt banting wants to work with a famo...,75,45,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,ns8803,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,R,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",20,158,min
8803,ns8804,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,TV-Y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",8,2,season
8804,ns8805,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,R,"comedies, horror movies",looking to survive in a world taken over by zo...,55,88,min
8805,ns8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,PG,"children & family movies, comedies","dragged from civilian life, a former superhero...",7,88,min


ETL is finished. next we will save the dataframe in a .csv file.

In [107]:
data_string.to_csv('Datasets/data_string.csv')